In [1]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler
 
import cv2
from matplotlib import pyplot as plt
import pickle
import datetime
import os 
import argparse

import argparse
import json
import numpy as np
import time
import os
import shutil
import psutil
import sys
import socket
import traceback

from collections import OrderedDict

import torch
from torch.utils.data import DataLoader

import robomimic
import robomimic.utils.train_utils as TrainUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.file_utils as FileUtils
from robomimic.config import config_factory
from robomimic.algo import algo_factory, RolloutPolicy
from robomimic.utils.log_utils import PrintLogger, DataLogger, flush_warnings

import matplotlib.pyplot as plt
import h5py

import os
import time 

from utils_ros import ROSInterface
import traceback

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/carl/robomimic/robomimic/scripts/setup_macros.py
)


In [2]:
import beepy as beep
beep_start = lambda : beep.beep('coin')

In [3]:
tb=Thread(target=beep_start)
tb.start()

In [4]:
device = TorchUtils.get_torch_device(try_to_use_cuda=True)

/home/carl/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
#14 march

ckpt_path1 = "/home/carl/data_sawyer/models/bcnd/spoon_c02/itr_6_model_0.pth.pth"
ckpt_path2 = "/home/carl/data_sawyer/models/bcnd/spoon_c02/itr_6_model_1.pth.pth"
ckpt_path3 = "/home/carl/data_sawyer/models/bcnd/spoon_c02/itr_6_model_2.pth.pth"

policy1, ckpt_dict1 = FileUtils.policy_from_checkpoint(ckpt_path=ckpt_path1, device=device, verbose=True)
policy2, ckpt_dict2 = FileUtils.policy_from_checkpoint(ckpt_path=ckpt_path2, device=device, verbose=True)
policy3, ckpt_dict3 = FileUtils.policy_from_checkpoint(ckpt_path=ckpt_path3, device=device, verbose=True)

============= Loaded Config =============
{
    "algo_name": "bc",
    "experiment": {
        "name": "spoon",
        "validate": false,
        "logging": {
            "terminal_output_to_txt": true,
            "log_tb": true,
            "log_wandb": false,
            "wandb_proj_name": "debug"
        },
        "save": {
            "enabled": true,
            "every_n_seconds": null,
            "every_n_epochs": 20,
            "epochs": [],
            "on_best_validation": false,
            "on_best_rollout_return": false,
            "on_best_rollout_success_rate": true
        },
        "epoch_every_n_steps": 500,
        "validation_epoch_every_n_steps": 50,
        "env": null,
        "additional_envs": null,
        "render": false,
        "render_video": true,
        "keep_all_videos": false,
        "video_skip": 5,
        "rollout": {
            "enabled": true,
            "n": 50,
            "horizon": 500,
            "rate": 20,
            "warmstart"

/home/carl/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/carl/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


============= Loaded Policy =============
ObservationKeyToModalityDict: mean not found, adding mean to mapping with assumed low_dim modality!
ObservationKeyToModalityDict: scale not found, adding scale to mapping with assumed low_dim modality!
ObservationKeyToModalityDict: logits not found, adding logits to mapping with assumed low_dim modality!
BC_GMM (
  ModuleDict(
    (policy): GMMActorNetwork(
        action_dim=7
        num_modes=1
        min_std=0.0001
        std_activation=softplus
        low_noise_eval=True
  
        encoder=ObservationGroupEncoder(
            group=obs
            ObservationEncoder(
                Key(
                    name=agentview_image
                    shape=[3, 84, 84]
                    modality=rgb
                    randomizer=CropRandomizer(input_shape=[3, 84, 84], crop_size=[76, 76], num_crops=1)
                    net=VisualCore(
                      input_shape=[3, 76, 76]
                      output_shape=[64]
                 

In [6]:
ckpt_dict = FileUtils.maybe_dict_from_checkpoint(ckpt_path=ckpt_path1, ckpt_dict=None)
# algo name and config from model dict
algo_name, _ = FileUtils.algo_name_from_checkpoint(ckpt_dict=ckpt_dict)
config, _ = FileUtils.config_from_checkpoint(algo_name=algo_name, ckpt_dict=ckpt_dict, verbose=False)

config.unlock()
config.train.data ="/home/carl/data_sawyer/spoon_pick/expert/spoon_pick_100_10d.hdf5"
config.lock()

ObsUtils.initialize_obs_utils_with_config(config)
dataset_path = os.path.expanduser(config.train.data)
shape_meta = FileUtils.get_shape_metadata_from_dataset(
    dataset_path=config.train.data,
    all_obs_keys=config.all_obs_keys,
    verbose=True
)


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['robot0_eef_pos']
using obs modality: rgb with keys: ['agentview_image', 'robot0_eye_in_hand_image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []
obs key agentview_image with shape (84, 84, 3)
obs key robot0_eef_pos with shape (6,)
obs key robot0_eye_in_hand_image with shape (84, 84, 3)


In [7]:
config.unlock()
config.train.data ="/home/carl/data_sawyer/spoon_pick/expert/spoon_pick_100_10d.hdf5"
config.lock()

In [8]:
def demo2actions(hdf5_file, demo_key, rollout_policy): 
    rollout_policy.start_episode()


    demo=hdf5_file['data'][demo_key]
    actions_real=demo['actions']
    obs=demo['obs']
    N=obs['robot0_eye_in_hand_image'].shape[0]

    obss=[]
    for i in range(N):
        ob={k:obs[k][i] for k in obs.keys()} 
        ob['robot0_eye_in_hand_image']=ob['robot0_eye_in_hand_image'].transpose(2,0,1)
        ob['agentview_image']=ob['agentview_image'].transpose(2,0,1)
        obss.append(ob)

    acs=[]
    for ob in obss: 
        ob = rollout_policy._prepare_observation(ob)
        ac = rollout_policy.policy.get_action(obs_dict=ob, goal_dict=None)
        ac=ac.detach().cpu().numpy()[0]
        acs.append(ac)

    actions_pred=np.array(acs)
    return actions_real, actions_pred

In [9]:
rclpy.init()

urdf_path = os.getcwd() +'/'+'sawyer.urdf'  #'/home/carl/sawyer_robot_ros2/src/teleop_script/sawyer.urdf'
robot = URDFModel(urdf_path)
jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}


joint_names=['right_j0', 'right_j1', 'right_j2', 'right_j3', 'right_j4', 'right_j5', 'right_j6']
joint_inds=[jointMap[name] for name in joint_names]


node = Node('infer_live')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)

t1 = Thread(target=ros_interface.spin_thread)
t1.start()

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

In [10]:
def get_ee(jointmsg):
    msg=jointmsg    
    q =  robot.getJoints() 
    for ind, name in enumerate(msg.name):
        if name in jointMap:
            q[jointMap[name]] = msg.position[ind]
    robot.setJoints(q)
    
    # T=robot.getBodyTransform('right_l6');
    T=robot.getBodyTransform('camera_link');
    xyz=T[0:3,3]
    e1 = rotm2euler.rotationMatrixToEulerAngles(T[0:3, 0:3])
    ee6=np.concatenate([xyz, e1])
    return ee6

def set_vels(vels):
    cmd_msg = JointCommand()
    cmd_msg.names = robot.jointNames
    cmd_msg.mode = cmd_msg.VELOCITY_MODE
    cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
    cmd_msg.velocity =vels 
    cmd_pub.publish(cmd_msg)

def goto_xyzrpy(target, delay=0.1, tol=0.05, max_step=100, gain=1.0, stop=True):
    ee6=get_ee(ros_interface.latest_joint_states)
    xd=ee6-target
    xd[3:6]=0 

    # print('xd: ', xd)

    alpha = .0001
    I = np.eye(q.shape[0])

    step=0
    while np.linalg.norm(xd)>tol:
        ee6=get_ee(ros_interface.latest_joint_states)
        xd=target-ee6 
        xd = xd * gain
        xd[3:6]=0        # only xyz, rpy not working yet

        J = robot.getJacobian('camera_link')
        
        qd = np.linalg.inv(J.T @ J + alpha * I) @ J.T @ xd

        set_vels(list(qd))
        time.sleep(delay)

        step = step+1 
        if step>=max_step:
            break 
    
    # stop
    if stop:
        set_vels([0.0]*10)

apos=0
def goto_q_pos_velocity_ctrl(target_q, alpha=0.7, tol=0.01,  delay=0.1, maxstep=50):
    global apos
    target_jointpos=target_q
    cpos=robot.getJoints()
    dpos=np.linalg.norm(target_jointpos-cpos)

    step =0
    while dpos>tol: 
        cpos=robot.getJoints()
        dpos=np.linalg.norm(target_jointpos-cpos)
          

        qd =(target_jointpos-cpos)*alpha
        apos=qd 
        if np.max(qd) <0.001:
            print('reached')
            break 


        set_vels(list(qd))
        time.sleep(delay)
        step +=1

        if step>=maxstep:
            break 
        
    set_vels([0.0]*10)

In [11]:
policies=[policy1, policy2, policy3] 

In [12]:
def predict_ensamble(policies, ob):
    acs=[]
    for rollout_policy in policies:
        ob2 = policy1._prepare_observation(ob)
        ac = policy1.policy.get_action(obs_dict=ob2, goal_dict=None)
        ac=ac.detach().cpu().numpy()[0]
        acs.append(ac)

    acts=np.vstack(acs)
    acts= np.mean(acts, axis=0)
    return acts 

### Robot safety check

In [13]:
pos1=robot.getJoints()
pos1

array([ 0.57911816,  0.21033203, -0.00692285, -1.6948291 ,  1.18522266,
       -1.55122852, -1.52731836,  2.56460156,  0.        ,  0.        ])

### move little by hand, then execute the following

In [10]:
goto_q_pos_velocity_ctrl(pos1, tol=0.005)

### spoon pick

In [14]:
ee_init_pos_spoon=np.array([ 0.21559961, -0.50414063,  0.31746387, -2.12591309,  1.28041895,
       -1.06813965, -1.98603027,  2.38494531,  0.04077414,  0.        ])

In [15]:
goto_q_pos_velocity_ctrl(ee_init_pos_spoon, tol=0.005)

'/home/carl/data_sawyer/spoon_hang/spoon_hang_10d_group.hdf5'

### Trajectory playing test

In [16]:
dataset_path =  config.train.data 
print(f'dataset_path: {dataset_path}')

f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())
len(demos)

dataset_path: /home/carl/data_sawyer/spoon_pick/expert/spoon_pick_100_10d.hdf5


100

In [17]:
demo_name="demo_1"
demo=f['data'][demo_name]
actions=demo['actions']
ee_pos=demo['obs']['robot0_eef_pos']
ee_pos.shape

(71, 6)

In [18]:
f['data'][demo_name]['obs'].keys()

<KeysViewHDF5 ['agentview_image', 'robot0_eef_pos', 'robot0_eye_in_hand_image', 'robot0_joint_pos', 'robot0_joint_vel']>

In [19]:
goto_xyzrpy(ee_pos[0], tol=0.005) 

In [17]:
for i in range(len(ee_pos)):
    goto_xyzrpy(ee_pos[i], delay=0.001, tol=0.02) 

goto_xyzrpy(ee_pos[i], tol=0.005)  # last one with precision

In [18]:
goto_xyzrpy(ee_pos[-1], tol=0.005)

In [20]:
eepos_current=ee_pos[0]
eepos_current

array([ 0.58733696, -0.03160624,  0.3367485 , -2.64198041,  1.55696198,
        2.13278455])

### Robot is ready

In [20]:
policy1.start_episode()
policy2.start_episode()
policy3.start_episode()

In [21]:
def get_current_obs():
    # img_wrist=cam_wrist.get_current_frame(scale=0.5)  
    # img_front=cam_front.get_current_frame(scale=0.5) 

    image_wrist=ros_interface.image_wrist
    image_front=ros_interface.image_front

    if image_front.sum()==0.0:
        print('image front None')
        return None
    if image_wrist.sum()==0:
        print('image wrist None')
        return None 

    scale=0.5
    img_wrist = cv2.resize(image_wrist, (int(image_wrist.shape[1]*scale), int(image_wrist.shape[0]*scale) ) , interpolation = cv2.INTER_AREA)
    img_front = cv2.resize(image_front, (int(image_front.shape[1]*scale), int(image_front.shape[0]*scale) ) , interpolation = cv2.INTER_AREA)


    # print('shape: ', image_wrist.shape, image_front.shape)
    msg=ros_interface.latest_joint_states
    gripper_isopen=ros_interface.latest_gripper_state

    ee=get_ee(msg)

    try:
        pos =np.array([msg.position[ind] for ind in joint_inds])
        vel =np.array([msg.velocity[ind] for ind in joint_inds])
        t=msg.header.stamp.sec+msg.header.stamp.nanosec*1e-9
    except Exception as e:
        # print('msg=', msg)
        # print('Error=',e, e.st)
        # traceback.print_exc()
        return None 

    img_wrist=cv2.cvtColor(img_wrist, cv2.COLOR_BGR2RGB)
    img_front=cv2.cvtColor(img_front, cv2.COLOR_BGR2RGB)

    img_wrist=cv2.resize(img_wrist, (84,84))
    img_front=cv2.resize(img_front, (84,84))


    ob={'agentview_image': img_front, 'robot0_eye_in_hand_image': img_wrist, 'robot0_joint_pos': pos, 'robot0_joint_vel':vel, 'robot0_eef_pos': ee}
    ob['robot0_eye_in_hand_image']=ob['robot0_eye_in_hand_image'].transpose(2,0,1)
    ob['agentview_image']=ob['agentview_image'].transpose(2,0,1)

    return ob

In [22]:
ob=get_current_obs()
ob!=None 

True

In [23]:
ob.keys()

dict_keys(['agentview_image', 'robot0_eye_in_hand_image', 'robot0_joint_pos', 'robot0_joint_vel', 'robot0_eef_pos'])

In [61]:
goto_xyzrpy(ee_pos[0], tol=0.005) 

In [26]:
eepos_start=ee_pos[0]
eepos_current=eepos_start
eepos_current

array([ 0.58733696, -0.03160624,  0.3367485 , -2.64198041,  1.55696198,
        2.13278455])

In [27]:
ob2 = policy1._prepare_observation(ob)
ac = policy1.policy.get_action(obs_dict=ob2, goal_dict=None)
ac=ac.detach().cpu().numpy()[0]
ac.shape

(7,)

In [28]:
ac=predict_ensamble(policies, ob)
ac.shape

(7,)

In [71]:
cv2.destroyAllWindows() 

In [70]:
goto_q_pos_velocity_ctrl(ee_init_pos_spoon, tol=0.005)

### try

In [69]:
frequency = 20.0   # similar to robomimic


eepos_start=ee_pos[0]
eepos_current=eepos_start


st=time.time()
acs=[]
for i in range(1000000):
    ob=get_current_obs()
    if ob==None:
        continue

    # ob2 = policy._prepare_observation(ob)
    # ac = policy.policy.get_action(obs_dict=ob2, goal_dict=None)
    # ac=ac.detach().cpu().numpy()[0]
    ac=predict_ensamble(policies, ob)
    print(f'i={i} ac={ac}')
    acs.append(ac)

    if ac[-1]>0.9:
        print('---------------------------------')
        print('grip now-----------------------')
        print('\n---------------------------------------')
        tb=Thread(target=beep_start)
        tb.start()
        # break 

    eepos_current[:3] += ac[:3]
    print(f'eepos_togo: {eepos_current}')

    goto_xyzrpy(eepos_current, tol=0.01, max_step=2, stop=False)
    


    image_front=ob['agentview_image'].transpose(2,1,0) 
    image_wrist=ob['robot0_eye_in_hand_image'].transpose(2,1,0)
    

    image=np.concatenate([image_wrist, image_front], axis=1) 
    image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imshow('frame',image)
    if cv2.waitKey(1) == 27: 
        print('UI closed')
        break  # esc to quit

    time.sleep(1.0/frequency)

cv2.destroyAllWindows() 

i=0 ac=[ 0.00023753  0.00331234 -0.0006048  -0.00909189 -0.00165825 -0.00802214
  0.10015032]
eepos_togo: [ 0.58757449 -0.0282939   0.3361437  -2.64198041  1.55696198  2.13278455]
i=1 ac=[ 8.44878741e-05  3.27188522e-03 -6.83060091e-04 -9.06281639e-03
 -1.66116096e-03 -8.05606600e-03  1.00028515e-01]
eepos_togo: [ 0.58765898 -0.02502201  0.33546064 -2.64198041  1.55696198  2.13278455]
i=2 ac=[ 0.00020063  0.0032586  -0.00069436 -0.00898823 -0.00156241 -0.00802664
  0.09998634]
eepos_togo: [ 0.58785961 -0.02176342  0.33476628 -2.64198041  1.55696198  2.13278455]
i=3 ac=[ 0.00032194  0.0032668  -0.00073974 -0.00902802 -0.00161903 -0.00804297
  0.10003808]
eepos_togo: [ 0.58818155 -0.01849662  0.33402654 -2.64198041  1.55696198  2.13278455]
i=4 ac=[ 0.00014223  0.00323477 -0.00075184 -0.01448327 -0.00178183 -0.01364581
  0.09974752]
eepos_togo: [ 0.58832378 -0.01526185  0.3332747  -2.64198041  1.55696198  2.13278455]
i=5 ac=[ 0.00013035  0.00325086 -0.00081737 -0.01154945 -0.00167814 -0.0

KeyboardInterrupt: 

In [38]:
goto_xyzrpy(ee_pos[0], tol=0.005)  

In [29]:
ac

array([-3.6969720e-04,  1.2310240e-03,  3.0301284e-04, -1.6575159e-03,
        4.3890024e-05, -1.8973439e-03, -2.0936792e-04], dtype=float32)

In [42]:
cv2.destroyAllWindows() 

### success: worked for the first time: (note: doesn't stop)

In [51]:
# eepos_current[:3] += ac 
# eepos_current

array([ 0.54482866,  0.15848914,  0.34229148, -0.0208837 ,  0.75747543,
       -1.54219383])

In [ ]:
goto_xyzrpy(ee_pos[0], tol=0.005) 

In [31]:
cv2.destroyAllWindows() 

In [36]:
acs2=np.vstack([acs])
acs2.shape

(57, 7)

In [37]:
np.sum(acs2[:,6])

1.0015614

In [38]:
acs2[:,6] > 0.8

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False])

In [37]:

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [38]:
acs2[:,6] 

array([-0.,  0., -0., -0., -0.,  0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0.,  0.,  0., -0., -0.,  0.,
       -0., -0., -0., -0., -0., -0.,  0., -0., -0., -0.,  0., -0., -0.,
       -0., -0., -0.,  0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        0.,  0.,  0., -0., -0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
      dtype=float32)

In [34]:
for i,ac in enumerate(acs2):
    if ac[-1]>0.9:
        print(i, 'grip now')

In [35]:
ac[-1]

-0.00024590347